In [1]:
import os
import json
import json
import regex as re
from tqdm.auto import tqdm
tqdm.pandas()
import math
import pandas as pd
import string
from pyvi.ViTokenizer import tokenize
import pandas as pd
import numpy as np
import json, pickle
from rank_bm25 import BM25Okapi
import argparse
import gc
from tqdm.auto import tqdm
tqdm.pandas()
import regex as re
import string

import json 
from glob import glob 
import re 
from nltk import word_tokenize as lib_tokenizer 
import string

from pandarallel import pandarallel
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, OkapiBM25Model
from gensim.similarities import SparseMatrixSimilarity
import numpy as np
pandarallel.initialize(progress_bar=True, use_memory_fs=False, nb_workers=10)

INFO: Pandarallel will run on 10 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.

https://nalepae.github.io/pandarallel/troubleshooting/


In [2]:
def post_process(x):
    x = " ".join(word_tokenize(strip_context(x))).strip()
    x = x.replace("\n"," ")
    x = "".join([i for i in x if i not in string.punctuation])
    return x

dict_map = dict({})  
def word_tokenize(text): 
    global dict_map 
    words = text.split() 
    words_norm = [] 
    for w in words: 
        if dict_map.get(w, None) is None: 
            dict_map[w] = ' '.join(lib_tokenizer(w)).replace('``', '"').replace("''", '"') 
        words_norm.append(dict_map[w]) 
    return words_norm 
 
 
def strip_context(text): 
    text = text.replace('\n', ' ') 
    text = re.sub(r'\s+', ' ', text) 
    text = text.strip() 
    return text

In [3]:
wiki_cleaned_path = "./processed/lagal_2024_10_31_cleaned_v2.csv"

In [4]:
df_wiki = pd.read_csv(wiki_cleaned_path)

In [5]:
df_wiki = df_wiki.fillna("NaN")

In [6]:
df_wiki.shape

(411210, 4)

In [7]:
all_bm25_texts = []
for i, row in tqdm(df_wiki.iterrows(), total=df_wiki.shape[0]):
    bm25_text = post_process(row["bm25_text"].lower())
    all_bm25_texts.append(bm25_text)
df_wiki["bm25_text"] = all_bm25_texts

  0%|          | 0/411210 [00:00<?, ?it/s]

In [8]:
df_wiki

,cid,text,bm25_text,i
0,0,"Thông tư này hướng dẫn tuần tra, canh gác bảo ...",thông tư này hướng dẫn tuần tra canh gác bảo v...,0
1,1,"1. Hàng năm trước mùa mưa, lũ, Ủy ban nhân dân...",1 hàng năm trước mùa mưa lũ ủy ban nhân dân cấ...,1
2,2,Tiêu chuẩn của các thành viên thuộc lực lượng ...,tiêu chuẩn của các thành viên thuộc lực lượng ...,2
3,3,"Nhiệm vụ của lực lượng tuần tra, canh gác đê\n...",nhiệm vụ của lực lượng tuần tra canh gác đê 1 ...,3
4,4,"Phù hiệu của lực lượng tuần tra, canh gác đê\n...",phù hiệu của lực lượng tuần tra canh gác đê ph...,4
...,...,...,...,...
411205,645443,phát sóng vô tuyến điện của Tổ chức Công ước t...,phát sóng vô tuyến điện của tổ chức công ước t...,261594
411206,645445,Chương VI. . ĐIỀU KHOẢN THI HÀNH\nĐiều 38. Quy...,chương vi điều khoản thi hành điều 38 quy định...,261595
411207,645445,"tục nhập khẩu.\n5. Cơ sở sản xuất, kinh doanh ...",tục nhập khẩu 5 cơ sở sản xuất kinh doanh hóa ...,261595
411208,645445,"tàng trữ, sử dụng và phá hủy vũ khí hóa học;\n...",tàng trữ sử dụng và phá hủy vũ khí hóa học b đ...,261595


In [ ]:
# df_wiki['bm25_text'] = df_wiki['bm25_text'].apply(lambda x: x.lower()).parallel_apply(post_process)
corpus = [x.split() for x in df_wiki['bm25_text'].values]
dictionary = Dictionary(corpus)
bm25_model = OkapiBM25Model(dictionary=dictionary)
bm25_corpus = bm25_model[list(map(dictionary.doc2bow, corpus))]
bm25_index = SparseMatrixSimilarity(bm25_corpus, num_docs=len(corpus), num_terms=len(dictionary),normalize_queries=False, normalize_documents=False)
tfidf_model = TfidfModel(dictionary=dictionary, smartirs='bnn')  # Enforce binary weighting of queries

In [10]:
os.makedirs("./processed/bm25_stage1",exist_ok=True)
dictionary.save("./processed/bm25_stage1/dict")
tfidf_model.save("./processed/bm25_stage1/tfidf")
bm25_index.save("./processed/bm25_stage1/bm25_index")